<a href="https://colab.research.google.com/github/perfectpanda-works/machine-learning/blob/master/PyTorchCNN_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#CIFAR10の読みこみ
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
訓練データの数
50000
テストデータの数
10000


In [8]:
#畳み込みニューラルネットワークの定義
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [9]:
#損失関数の定義
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [10]:
#GPU化するためにはこのブロックを実行
#ColaboのハードウェアアクセラレータもGPUに変更
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)

cuda:0


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [15]:
#訓練
for epoch in range(2):  #チュートリアルでは2エポック訓練を行う。

    running_loss = 0.0
    #enumerateでiにリスト番号、dataに内容が入る。
    for i, data in enumerate(trainloader, 0):
        #data変数は[inputs, labels]のリストデータ。
        #inputs, labels = data #CPUのとき
        inputs, labels = data[0].to(device), data[1].to(device) #GPU

        #勾配のパラメータを初期化
        optimizer.zero_grad()

        #順伝播し、逆伝播して重みを更新
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    #ミニバッチ2000ごとに状況の表示
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        
print('Finished Training')

[1,  2000] loss: 0.942
[1,  4000] loss: 0.943
[1,  6000] loss: 0.967
[1,  8000] loss: 0.947
[1, 10000] loss: 0.971
[1, 12000] loss: 1.001
[2,  2000] loss: 0.911
[2,  4000] loss: 0.927
[2,  6000] loss: 0.951
[2,  8000] loss: 0.926
[2, 10000] loss: 0.970
[2, 12000] loss: 0.965
[3,  2000] loss: 0.868
[3,  4000] loss: 0.897
[3,  6000] loss: 0.929
[3,  8000] loss: 0.927
[3, 10000] loss: 0.940
[3, 12000] loss: 0.962
[4,  2000] loss: 0.890
[4,  4000] loss: 0.880
[4,  6000] loss: 0.897
[4,  8000] loss: 0.889
[4, 10000] loss: 0.934
[4, 12000] loss: 0.941
[5,  2000] loss: 0.847
[5,  4000] loss: 0.857
[5,  6000] loss: 0.890
[5,  8000] loss: 0.907
[5, 10000] loss: 0.900
[5, 12000] loss: 0.929
[6,  2000] loss: 0.850
[6,  4000] loss: 0.848
[6,  6000] loss: 0.874
[6,  8000] loss: 0.890
[6, 10000] loss: 0.901
[6, 12000] loss: 0.919
[7,  2000] loss: 0.812
[7,  4000] loss: 0.842
[7,  6000] loss: 0.873
[7,  8000] loss: 0.888
[7, 10000] loss: 0.917
[7, 12000] loss: 0.892
[8,  2000] loss: 0.821
[8,  4000] 

In [18]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        #images, labels = data #CPU
        images, labels = data[0].to(device), data[1].to(device) #GPU
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 61 %


In [19]:
#モデルの保存
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)